In [1]:
!pip install -q accelerate -U
!pip install -q bitsandbytes -U
!pip install -q trl -U
!pip install -q peft -U
!pip install -q transformers -U

In [2]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [3]:
from datasets import load_dataset

dataset = load_dataset("HuggingFaceH4/ultrachat_200k", trust_remote_code=True, split="train_sft")
dataset = dataset.shuffle(seed=0).select(range(10_000))

README.md:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

(…)-00000-of-00003-a3ecf92756993583.parquet:   0%|          | 0.00/244M [00:00<?, ?B/s]

(…)-00001-of-00003-0a1804bcb6ae68c6.parquet:   0%|          | 0.00/244M [00:00<?, ?B/s]

(…)-00002-of-00003-ee46ed25cfae92c6.parquet:   0%|          | 0.00/244M [00:00<?, ?B/s]

(…)-00000-of-00001-f7dfac4afe5b93f4.parquet:   0%|          | 0.00/81.2M [00:00<?, ?B/s]

...

(…)-00000-of-00001-3d4cd8309148a71f.parquet:   0%|          | 0.00/80.4M [00:00<?, ?B/s]

Generating train_sft split:   0%|          | 0/207865 [00:00<?, ? examples/s]

Generating test_sft split:   0%|          | 0/23110 [00:00<?, ? examples/s]

Generating train_gen split:   0%|          | 0/256032 [00:00<?, ? examples/s]

Generating test_gen split:   0%|          | 0/28304 [00:00<?, ? examples/s]

In [4]:
from transformers import AutoTokenizer

#template_tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0") 
# TinyLlama/TinyLlama-1.1B-Chat-v1.0: We use this Tokinizer from this chat intr just to use the 
# method apply_chat_template auto apply chat msg template

# format_prompt Method use to convert in chat format
#def format_prompt(example):
#  """Format the prompt using the <|user|> and <|assistant|> format"""
#  chat = example['messages']
#  prompt = template_tokenizer.apply_chat_template(chat, tokenize=False)

#  return {'text': prompt}

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [5]:
#print(format_prompt(dataset[0])['text'])

In [6]:
#dataset = dataset.map(format_prompt)

In [7]:
#print(dataset[0]['text'])

### Testing Base LLAMA Model
- Let's see how base Tiny-LLAMA performs out of the box

In [8]:
from transformers import pipeline

# base model
model_name = "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T"

pipe = pipeline(task='text-generation', model=model_name, device='cuda')


# prompt
# <|user|>, <|assistant|>

prompt = """<|user|>
Tell me something about Large Language Models.</s>
<|assistant|>
"""

prompt = """
Tell me something about Large Language Models
"""

output = pipe(prompt)
output

config.json:   0%|          | 0.00/560 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.40G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

[{'generated_text': '\nTell me something about Large Language Models\n\nA: I think the best way to understand LLMs is to look at the code.\nThe code is available on GitHub, and you can find the code for the BERT model here.\nThe code for the RoBERTa model is here.\nThe code for the XLM model is here.\nThe code for the XLM-R model is here.\nThe code for the XLM-R model is here.\nThe code for the XLM-R model is here.\nThe code for the XLM-R model is here.\nThe code for the XLM-R model is here.\nThe code for the XLM-R model is here.\nThe code for the XLM-R model is here.\nThe code for the XLM-R model is here.\nThe code for the XLM-R model is here.\nThe code for the XLM-R model is here.\nThe code for the XLM-R model is here.\nThe code for the XLM-R model is here.\nThe code for the XLM-R model is here.\nThe code for the XLM-R model is here.\nThe code for the XLM-R model is here.\nThe code for the XLM-R model is here.\nThe code for the XLM-R model is here.\nThe code for the XLM-R model is he

Note: Base model can't provide an answer

## Model Configuration for Training

In [9]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig



In [10]:
# do the  4-bit quantization configuration in Q-LORA
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype='float16',
    bnb_4bit_use_double_quant=True
)

In [11]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = "<PAD>"
tokenizer.padding_size="left"

tokenizer.chat_template = """
{% for message in messages %}
{% if message['role'] == 'user' %}
<|user|>\n{{ message['content'] }}</s>
{% elif message['role'] == 'assistant' %}
<|assistant|>\n{{ message['content'] }}</s>
{% endif %}
{% endfor %}
""".strip()

In [12]:
example = dataset[0]
chat = example['messages']
prompt = tokenizer.apply_chat_template(chat, tokenize=False)
print(prompt)

<|user|>
3. Heat the vegetables in boiling water for the recommended time.</s>
<|assistant|>
I do not have information about the specific type of vegetables being referred to. However, here are general instructions for boiling most vegetables:

1. Wash the vegetables thoroughly with clean water.
2. Cut the vegetables into small or medium-sized pieces.
3. Bring a pot of water to boil on the stove.
4. Add a pinch of salt to the boiling water.
5. Add the vegetables to the boiling water.
6. Cook the vegetables for the recommended time (see cooking instructions on the package or look up cooking times for specific vegetables online).
7. Test the vegetables for doneness using a fork or a knife. They should be tender but not overcooked and mushy.
8. Once the vegetables are cooked to your desired tenderness, remove them from the boiling water using a slotted spoon or a strainer.
9. Drain the vegetables and serve hot with your favorite seasonings or sauce.</s>
<|user|>
Can you add some informati

In [13]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map = "auto",
    quantization_config=bnb_config
)

In [14]:
model.config.use_cache=False
model.config.pretraining_tp=1

# Prepare LoRA Configuration for PEFT Fine tuning


In [15]:
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

peft_config = LoraConfig(
    lora_alpha=32,
    lora_dropout=0.1,
    r=64,
    bias='none',
    task_type='CAUSAL_LM',
    target_modules=['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj']
)

model = prepare_model_for_kbit_training(model)

model = get_peft_model(model, peft_config)

In [16]:
w = 2048*256
a = 2048*64
b = 64*256

w, a, b, a+b, (a+b)/w

(524288, 131072, 16384, 147456, 0.28125)

## Model Fine Tuning

In [17]:
from transformers import TrainingArguments, Trainer
from trl import SFTTrainer, SFTConfig

output_dir = "train_dir"

args = SFTConfig(
    output_dir=output_dir,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    num_train_epochs=1,
    logging_steps=10,
    fp16=True,
    gradient_checkpointing=True
)

trainer = SFTTrainer(
    model=model,
    train_dataset = dataset,
    dataset_text_field='messages',
    tokenizer = tokenizer,
    args=args,
    max_seq_length=512,
    peft_config = peft_config
)

trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/opt/

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss
10,1.600500
20,1.416400
30,1.478800
40,1.548600
50,1.572200
60,1.362500
70,1.468700
80,1.425400
90,1.545100
100,1.353100


/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


TrainOutput(global_step=1250, training_loss=1.3697360481262206, metrics={'train_runtime': 7030.1694, 'train_samples_per_second': 1.422, 'train_steps_per_second': 0.178, 'total_flos': 3.3318873700171776e+16, 'train_loss': 1.3697360481262206, 'epoch': 1.0})

In [18]:
trainer.model.save_pretrained("TinyLlama-1.1B-qlora")

## Load Pre-Trained PEFT Model for Prediction

In [19]:
from peft import AutoPeftModelForCausalLM

model = AutoPeftModelForCausalLM.from_pretrained(
    "TinyLlama-1.1B-qlora",
    device_map='auto'
)

merged_model = model.merge_and_unload()

In [20]:
from transformers import pipeline

prompt = """<|user|>
Tell me something about Large Language Models.</s>
<|assistant|>
"""

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = "<PAD>"
tokenizer.padding_size="left"

pipe = pipeline(task='text-generation', model=merged_model, tokenizer=tokenizer)
output = pipe(prompt)
print(output[0]['generated_text'])

<|user|>
Tell me something about Large Language Models.</s>
<|assistant|>
Large Language Models (LLMs) are a type of artificial intelligence (AI) that can generate human-like language. They are trained on large amounts of text data, and they can be used to generate text in a variety of languages. LLMs are becoming increasingly popular in various fields, including natural language processing (NLP), machine translation, and chatbots.

One of the key advantages of LLMs is their ability to generate human-like language. They can generate text that is grammatically correct, spelling correct, and has a natural flow and rhythm. This makes them ideal for use in chatbots, where they can provide helpful information or answer questions quickly and accurately.

Another advantage of LLMs is their ability to learn from experience. They can be trained to generate different types of text based on the context of the input text. This means that they can generate different types of text based on the conte